In [2]:
import sys
sys.path.append("..")
import torch
from torch_geometric.utils.convert import to_networkx
from Openseas_Methods import *
from torch_geometric.loader import DataLoader
from graph_utils import *
from network_utils import *
from GraphDataset import GraphDataset
from tqdm import tqdm

In [5]:
dataset = GraphDataset('dataset_stor/graph_dataset_3',normalize=True)

19934
19835


In [18]:
dataset.get(0)

Data(x=[1706317, 1], edge_index=[2, 6660074], y=[1706317], collection=[1706317], label=[1706317], training_mask=[1706317], test_mask=[1706317], val_mask=[1706317])

In [6]:
sum(list(dataset.get(7).collection))

tensor(697)

In [25]:
graph = to_networkx(dataset.get(0), node_attrs=['collection', 'label', 'training_mask', 'test_mask', 'val_mask'])

In [26]:
print(graph.nodes[0]['collection'])

True


In [3]:
dataset4 = GraphDataset('dataset_stor/graph_dataset_4',normalize=True)
sum(list(dataset4.get(7).collection))

12381
12282


tensor(1044)

In [4]:
dataset4.get(7)

Data(x=[1989540, 1], edge_index=[2, 8940286], y=[1989540], collection=[1989540], label=[1989540], training_mask=[1989540], test_mask=[1989540], val_mask=[1989540])

In [60]:
# Compute graph-wide metrics once
# harmonic = nx.harmonic_centrality(graph)
# print('harmonic centrality completed')
# eigenvector = nx.eigenvector_centrality_numpy(graph)
# print('eigenvector centrality completed')
# pagerank = nx.pagerank(graph)
# print('computed graph-wide metrics')

# Function to compute node-specific metrics
def compute_node_specific_metrics(G, node):
    metrics = {}

    # start_time = time.time()
    # Computing each metric and recording the time taken
    metrics["degree"] = G.degree(node)
    # print('degree', time.time() - start_time)

    # start_time = time.time()
    metrics["clustering_coefficient"] = nx.clustering(G, node)
    # print("clustering coefficient", time.time() - start_time)

    # start_time = time.time()
    # NOTE: closeness is super slow
    # metrics["closeness_centrality"] = nx.closeness_centrality(G, node)
    # print("closeness centrality", time.time() - start_time)

    # start_time = time.time()
    metrics["square_clustering"] = nx.square_clustering(G, node)
    # print("square_clustering", time.time() - start_time)

    return metrics

# Assigning metrics as node attributes for the subset of collection nodes
collection_nodes = [node for node in graph if graph.nodes[node]['collection']]
for node in tqdm(graph):
    if graph.nodes[node]['collection']:
        node_metrics = compute_node_specific_metrics(graph, node)
    # node_metrics["harmonic_centrality"] = harmonic[node]
    node_metrics["eigenvector_centrality"] = eigenvector[node]
    # node_metrics["pagerank"] = pagerank[node]
    graph.nodes[node]['metrics'] = node_metrics



MemoryError: Unable to allocate 21.2 TiB for an array with shape (1706317, 1706317) and data type int64

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import numpy as np

# Assuming each node in the graph has 'label', 'train', 'val', 'test' attributes
# Extracting features (metrics) and labels
features = []
labels = []
train_mask = []
val_mask = []
test_mask = []

for node in graph.nodes(data=True):
    features.append(list(node[1]['metrics'].values()))
    labels.append(node[1]['label'])
    train_mask.append(node[1]['train_mask'])
    val_mask.append(node[1]['val_mask'])
    test_mask.append(node[1]['test_mask'])

# Converting lists to PyTorch tensors
features = torch.tensor(features, dtype=torch.float32)
labels = torch.tensor(labels, dtype=torch.long)
train_mask = torch.tensor(train_mask, dtype=torch.bool)
val_mask = torch.tensor(val_mask, dtype=torch.bool)
test_mask = torch.tensor(test_mask, dtype=torch.bool)

# Define a custom dataset
class GraphDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Creating datasets
train_dataset = GraphDataset(features[train_mask], labels[train_mask])
val_dataset = GraphDataset(features[val_mask], labels[val_mask])
test_dataset = GraphDataset(features[test_mask], labels[test_mask])

# DataLoader for batching
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Initialize the MLP
input_size = len(features[0])  # Number of input features
hidden_size = 64  # You can tune this
num_classes = len(torch.unique(labels))  # Number of unique labels
model = MLP(input_size, hidden_size, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training the MLP
def train(model, data_loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for data in data_loader:
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(data_loader)

# Validation
def validate(model, data_loader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for data in data_loader:
            inputs, labels = data
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
    return total_loss / len(data_loader)

# Training loop
num_epochs = 20  # You can tune this
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer, criterion)
    val_loss = validate(model, val_loader, criterion)
    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')

# We can also evaluate the model on the test set after training
# However, for brevity, I'm not including that here. You can use the validate function for testing as well.

# Note: This is a basic implementation. Depending on your dataset, you might need to adjust the model architecture, hyperparameters, etc.
